In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.2MB/s 
     |████████████████████████████████| 2.9MB 24.9MB/s 
     |████████████████████████████████| 890kB 33.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=416ebca9f8928cdfaf2d8ab0e834479957a146191f61e682fd39aee3270ee2e7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import pandas as pd
import json
from transformers import BertTokenizer,TFBertModel
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
import random
import nltk
from nltk.tokenize import word_tokenize

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!cp "/content/drive/My Drive/toxic span/toxic_span_practice.csv" "./toxic_span_practice.csv"
!cp "/content/drive/My Drive/toxic span/toxic_span_train.csv" "./toxic_span_train.csv"

In [5]:
def f1(preds,trues):
    if len(trues) == 0:
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [6]:
def avg_f1(preds,trues):
    avg_f1_total = 0.0
    for pred,true in zip(preds,trues):
        avg_f1_total += f1(pred,true)
    return avg_f1_total/len(preds)

In [7]:
test_set = pd.read_csv("toxic_span_practice.csv")
test_set['spans'] = test_set['spans'].apply(lambda x : json.loads(x))
train_set = pd.read_csv("toxic_span_train.csv")
train_set['spans'] = train_set['spans'].apply(lambda x : json.loads(x))
dataset = test_set.append(train_set,ignore_index=True)
dataset['text'] = dataset['text'].apply(lambda x : x.lower())

# random base 

## tokenize using spaces


In [ ]:
indices_all = []
for index,row in dataset.iterrows():
    text = row['text']
    start = 0
    indices = []
    for word in text.split():
        if random.choice([True,False]):
            start_index = text[start:].find(word)
            indices += list(range(start + start_index,start + start_index+len(word)))
            start += start_index+len(word)
    indices_all.append(indices)

In [ ]:
toxic_f1 = avg_f1(indices_all,dataset['spans'].to_numpy())

In [ ]:
print("F1 : %f"%(toxic_f1))

F1 : 0.173530


## tokenizing using nltk

In [13]:
indices_all = []
for index,row in dataset.iterrows():
    text = row['text']
    start = 0
    indices = []
    for word in word_tokenize(text):
        if random.choice([True,False]):
            start_index = text[start:].find(word)
            indices += list(range(start + start_index,start + start_index+len(word)))
            start += start_index+len(word)
    indices_all.append(indices)

In [14]:
toxic_f1 = avg_f1(indices_all,dataset['spans'].to_numpy())

In [15]:
print("F1 : %f"%(toxic_f1))

F1 : 0.179100


# cluster words with bert as embedding --- not working right now

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
kf = KFold(n_splits=10)
results_train = []
results_test = []
dataset_texts = dataset['text'].to_numpy()
dataset_targets = dataset[].to_numpy()
for train_index,test_index in kf.split(data_test):
    train_data,test_data = dataset_texts[train_index],dataset_texts[test_index]
    train_target,test_target = dataset_targets[train_index],dataset_targets[test_index]
    knn = KMeans(n_splits=2)
    input_train = tokenizer(train_data)
    model_prediction = model(input_train)
    knn.fit(model_prediction)
    input_test = tokenizer(test_data)
    model_prediction = model(input_test)
    prediction_knn = knn.predict(model_prediction)
    